In [1]:
""" Solver. 
Takes trials and infers value of symbols.
"""

' Solver. \nTakes trials and infers value of symbols.\n'

In [2]:
import numpy as np
import random
from functions.rnn_cryptic import generate_sequences, convert_seq2inputs,\
                                  pad_seqs_2step, pad_seqs_1step, calculate_output, onehot2seq 


In [3]:
def seq2str(seqs):
    A = []
    B = []
    for seq in seqs:
        a = ''
        for elem in seq:
            if type(elem) is str:
                a+= elem
            elif type(elem) is tuple:
                for el in elem:
                    a+=el
            else:
                B.append(elem)
        A.append(a)
            
    return A, B

def coef_mat(trials, rows, cols, unique_chars):
    res = np.zeros([rows, cols])
    for i, t in enumerate(trials):
        op = 1
        for e in t:
            if e == '+':
                op = 1
            elif e == '-':
                op = -1
            else:
                res[i, unique_chars.index(e)] += op*1
    return res

# def get_mats(seqs):
    
#     trials, ans = seq2str(seqs)
#     tot_chars = ''.join(ch for ch in ''.join(trials) if ch.isalnum())
#     unique_chars = list(set(tot_chars))
#     rows = len(seqs)
#     cols = len(unique_chars)
#     res = coef_mat(trials, rows, cols, unique_chars)
    
#     return res, ans

def get_rank(seqs):
    
    trials, ans = seq2str(seqs)
    tot_chars = ''.join(ch for ch in ''.join(trials) if ch.isalnum())
    unique_chars = list(set(tot_chars))
    rows = len(seqs)
    cols = len(unique_chars)
    res = coef_mat(trials, rows, cols, unique_chars)
    
    rank = np.linalg.matrix_rank(res)
    print('rank is: ', rank, ' no. unknowns is: ', cols)
    if rank == cols:
        print('matrix is determined\n')
        out = 1
    elif rank < cols:
        print('matrix is underdetermined\n')
        out = 0
    return out

def change_padder(seqs, cue_dict, xval):
    """ changes value of X from zero to a new value
        and recalculates sequence output"""
    cue_dict['X'] = xval
    inps = [s[:-1] for s in seqs]
    for inp in inps:
        inp.append(calculate_output(inp, cue_dict, bidmas = False))

    return inps, cue_dict

          

In [4]:
num_inputs = 2

###############################
total_syms = ['X','A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
all_syms = total_syms[:num_inputs]
all_input_vals = list(np.arange(2,18))
input_vals = random.sample(all_input_vals,num_inputs)
# randomly select values for each input
cue_dict = {}
for i, s in enumerate(all_syms):
    cue_dict[s] = input_vals[i]

# generate trials
p_add = pad_seqs_1step(all_syms, cue_dict)
p_sub = pad_seqs_1step(all_syms, cue_dict, padder=('-','X'))
train_sims = all_syms[:-1]
trainseqs_add = generate_sequences(['+'], train_sims, len_seq = 2, cue_dict = cue_dict, init_values = train_sims)
testseqs_add = generate_sequences(['+'], all_syms, len_seq = 2, cue_dict = cue_dict, init_values = all_syms)
trainseqs_sub = generate_sequences(['-'], train_sims, len_seq = 2, cue_dict = cue_dict, init_values = train_sims)
testseqs_sub = generate_sequences(['-'], all_syms, len_seq = 2, cue_dict = cue_dict, init_values = all_syms)

# remove any trian trials from test set
for val in trainseqs_add:
    if val in testseqs_add:
        testseqs_add.remove(val)
for val in trainseqs_sub:
    if val in testseqs_sub:
        testseqs_sub.remove(val)

# randomly select a test trial to train non_primitive on
rand_trials = random.sample(testseqs_add, 2)
trainseqs_add_np = trainseqs_add + rand_trials
for v in rand_trials:
    testseqs_add.remove(v)
rand_trials = random.sample(testseqs_sub, 2)
trainseqs_sub_np = trainseqs_sub + rand_trials
for v in rand_trials:
    testseqs_sub.remove(v)


In [5]:
cue_dict

{'X': 2, 'A': 15}

In [6]:
get_rank(trainseqs_add + p_add)

rank is:  2  no. unknowns is:  2
matrix is determined



1

In [7]:
print('primitive training subtraction, with no. symbols = ', num_inputs)
get_rank(pad_seqs_2step(trainseqs_sub, padder=('-','X')) + p_sub)

print('primitive training addition, with no. symbols = ', num_inputs)
get_rank(pad_seqs_2step(trainseqs_add, padder=('+','X')) + p_sub)

print('without primitive training subtraction, with no. symbols = ', num_inputs)
get_rank(pad_seqs_2step(trainseqs_sub_np, padder=('-','X')))

print('without primitive training addition, with no. symbols = ', num_inputs)
get_rank(pad_seqs_2step(trainseqs_add_np, padder=('+','X')))

primitive training subtraction, with no. symbols =  2
rank is:  2  no. unknowns is:  2
matrix is determined

primitive training addition, with no. symbols =  2
rank is:  2  no. unknowns is:  2
matrix is determined

without primitive training subtraction, with no. symbols =  2
rank is:  2  no. unknowns is:  2
matrix is determined

without primitive training addition, with no. symbols =  2
rank is:  2  no. unknowns is:  2
matrix is determined



1